In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from custom_models import cnn
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import time
import os
from PIL import Image
from torchvision.transforms import v2
from tempfile import TemporaryDirectory
import shutil
import random

cudnn.benchmark = True
plt.ion()   # interactive mode

#### Data Transformations

In [2]:
# data transformations to loop through
minimal_transforms = {
    'synthetic_train': transforms.Compose([
        v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
        v2.Grayscale(num_output_channels=1),
        v2.Normalize([0.5, ],[0.5, ]),
        v2.Resize((128, 128))
    ]),
    'test': transforms.Compose([
        v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
        v2.Grayscale(num_output_channels=1),
        v2.Normalize([0.5, ],[0.5, ]),
        v2.Resize((128, 128))
    ]),
}

basic_transforms = {
    'synthetic_train': transforms.Compose([
        v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
        v2.Grayscale(num_output_channels=1),
        v2.Normalize([0.5, ],[0.5, ]),
        v2.RandomHorizontalFlip(p=0.5),
        v2.Resize((128, 128))
    ]),
    'test': transforms.Compose([
        v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
        v2.Grayscale(num_output_channels=1),
        v2.Normalize([0.5, ],[0.5, ]),
        v2.Resize((128, 128))
    ]),
}

auto_transforms = {
    'synthetic_train': transforms.Compose([
        v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
        v2.Grayscale(num_output_channels=1),
        v2.Normalize([0.5, ],[0.5, ]),
        v2.AutoAugment(policy=v2.AutoAugmentPolicy.IMAGENET),
        v2.Resize((128, 128))
    ]),
    'test': transforms.Compose([
        v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)]),
        v2.Grayscale(num_output_channels=1),
        v2.Normalize([0.5, ],[0.5, ]),
        v2.Resize((128, 128))
    ]),
}

#### Create Training Set with Real/Synthetic Images

In [3]:
def makeSyntheticTrain(train_directory, synthetic_train_directory, train_percentage, synthetic_percentage):

    # Remove any existing images in directory
    try:
        shutil.rmtree(synthetic_train_directory)
    except:
        print("directory does not exist")

    # Loop through subfolders, generate synthetic images
    subfolders = [f for f in os.listdir(train_directory)]

    for s in subfolders:
        # for each subfolder in the train directory, make the same in the synthetic train directory
        os.makedirs(f"{synthetic_train_directory}/{s}", exist_ok=True)
        
        # get a random sample from each subfolder
        subfolder_path = f"{train_directory}/{s}"
        files = os.listdir(subfolder_path)
        sample_files = random.sample(files, round(len(files)*train_percentage))
        
        # create synthetic sample based on sampled original images
        synthetic_subfolder_path = subfolder_path.replace('train','synthetic')
        synthetic_files = [f for f in os.listdir(synthetic_subfolder_path) if int(f.replace('.png','').split('_')[1]) in [int(f.replace('.png','').split('_')[1]) for f in sample_files]]
        synthetic_sample_files = random.sample(synthetic_files, round(len(files)*synthetic_percentage))
        
        # Move sample files to synthetic directory
        for f in sample_files:
            
            image_path = f"{subfolder_path}/{f}"
            destination_directory = f"{synthetic_train_directory}/{s}/"
            shutil.copyfile(image_path, destination_directory + image_path.split('/')[-1])

        # Move synthetic sample files to synthetic directory
        for f in synthetic_sample_files:

            image_path = f"{synthetic_subfolder_path}/{f}"
            destination_directory = f"{synthetic_train_directory}/{s}/"
            shutil.copyfile(image_path, destination_directory + image_path.split('/')[-1])

#### Read Data

In [4]:
def get_data(data_dir, data_sets, data_transforms):
    
    image_datasets = {
        x: datasets.ImageFolder(
            os.path.join(data_dir, x),
            data_transforms[x]
        )
        for x in data_sets
    }

    dataloaders = {
        x: DataLoader(
            image_datasets[x], 
            batch_size=4,
            shuffle=True, 
            num_workers=4
        )
        for x in data_sets
    }

    dataset_sizes = {
        x: len(image_datasets[x]) 
        for x in data_sets
    }

    class_names = image_datasets['synthetic_train'].classes

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    return image_datasets, dataloaders, dataset_sizes, class_names, device

#### Train Model

In [5]:
def train_model(model, criterion, optimizer, dataloaders, dataset_sizes, num_epochs=10):
    since = time.time()

    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        
        best_test_loss = float('inf')
        patience = 2  # Number of epochs to wait for improvement before stopping
        test_losses = []
        train_losses = []
        test_acc = []
        train_acc = []
        patience_counter = 0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['synthetic_train', 'test']:
                if phase == 'synthetic_train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'synthetic_train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'synthetic_train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                
                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double().item() / dataset_sizes[phase]

                if phase =='synthetic_train':
                    train_losses.append(epoch_loss)
                    train_acc.append(epoch_acc)
                else:
                    test_losses.append(epoch_loss)
                    test_acc.append(epoch_acc)
                    
                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if phase == 'test' and epoch_loss <= best_test_loss:
                    best_test_loss = epoch_loss
                    patience_counter = 0  # Reset counter
                    torch.save(model.state_dict(), best_model_params_path)
                elif phase == 'test' and epoch_loss > best_test_loss:
                    patience_counter += 1
                
                # Early stopping check
                #if patience_counter >= patience:
                #    print("Stopping early due to no improvement in validation loss.")
                #    break

        # store results in dataframe
        dat = {
            "epoch": range(len(test_losses)),
            "test_losses": test_losses,
            "train_losses": train_losses,
            "test_accuracies": test_acc,
            "train_accuracies": train_acc
        }

        result = pd.DataFrame(data=dat)
        print()
        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best test Loss: {best_test_loss:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path, weights_only=True))
    return result, model

#### Loop through different training scenarios

In [6]:
num_epochs = 10
num_sims = 10
train_percs = [0.6, 0.6, 0.4, 0.4]
synth_percs = [0.4, 0.0, 0.6, 0.0]
transforms = {
    'minimal': minimal_transforms,
    'basic': basic_transforms,
    'auto': auto_transforms
}

for train_percentage, synthetic_percentage in zip(train_percs, synth_percs):
    for active_transform in transforms.keys():
        df_all_results = pd.DataFrame()
        for n in range(num_sims):
    
            # make the synthetic training dataset
            makeSyntheticTrain(
                train_directory='../data/alzheimer_mri/train',
                synthetic_train_directory='../data/alzheimer_mri/synthetic_train', 
                train_percentage=train_percentage, 
                synthetic_percentage=synthetic_percentage
            )

            # get and load datasets
            data_dir = '../data/alzheimer_mri'
            data_sets = ['synthetic_train','test']
            image_datasets, dataloaders, dataset_sizes, class_names, device = get_data(
                data_dir=data_dir, 
                data_sets=data_sets, 
                data_transforms=transforms.get(active_transform)
            )

            # instantiate model
            model = cnn.CNN(in_channels=1, num_classes=4)

            # Set the size of each output sample to nn.Linear(num_ftrs, len(class_names))
            #num_ftrs = 4 #model.fc.in_features
            #model.fc = nn.Linear(num_ftrs, len(class_names))
            model = model.to(device)

            # train the model
            df_results,_ = model = train_model(
                model=model, 
                criterion = nn.CrossEntropyLoss(),
                optimizer = optim.Adam(model.parameters(), lr=0.001), 
                dataloaders=dataloaders,
                dataset_sizes=dataset_sizes,
                num_epochs=num_epochs
            )

            df_results['train_percentage'] = train_percentage
            df_results['synth_percentage'] = synthetic_percentage
            df_results['transform'] = active_transform
            df_results['sim_num'] = n
            df_results['category'] = df_results.apply(lambda row: row['transform']+'_'+str(row['train_percentage'])+'_'+str(row['synth_percentage']), axis=1)
            df_all_results = pd.concat([df_all_results, df_results],ignore_index=True)
        df_all_results.to_csv('../results/results_cnn_'+active_transform+'_'+str(train_percentage)+'_'+str(synthetic_percentage)+'.csv')

/home/danielkwon/MAS-thesis/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 0/9
----------
synthetic_train Loss: 0.9825 Acc: 0.5244
test Loss: 0.9504 Acc: 0.5398
Epoch 1/9
----------
synthetic_train Loss: 0.8586 Acc: 0.5859
test Loss: 0.7173 Acc: 0.6836
Epoch 2/9
----------
synthetic_train Loss: 0.6380 Acc: 0.7174
test Loss: 0.4553 Acc: 0.8117
Epoch 3/9
----------
synthetic_train Loss: 0.3771 Acc: 0.8535
test Loss: 0.4536 Acc: 0.8438
Epoch 4/9
----------
synthetic_train Loss: 0.1826 Acc: 0.9330
test Loss: 0.4373 Acc: 0.8781
Epoch 5/9
----------
synthetic_train Loss: 0.0885 Acc: 0.9699
test Loss: 0.4724 Acc: 0.8641
Epoch 6/9
----------
synthetic_train Loss: 0.0536 Acc: 0.9828
test Loss: 0.4413 Acc: 0.9148
Epoch 7/9
----------
synthetic_train Loss: 0.0567 Acc: 0.9814
test Loss: 0.4097 Acc: 0.9172
Epoch 8/9
----------
synthetic_train Loss: 0.0411 Acc: 0.9871
test Loss: 0.4209 Acc: 0.9180
Epoch 9/9
----------
synthetic_train Loss: 0.0312 Acc: 0.9902
test Loss: 0.6026 Acc: 0.8938

Training complete in 3m 30s
Best test Loss: 0.409655


/home/danielkwon/MAS-thesis/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 0/9
----------
synthetic_train Loss: 0.9874 Acc: 0.5170
test Loss: 0.8495 Acc: 0.5859
Epoch 1/9
----------
synthetic_train Loss: 0.7878 Acc: 0.6355
test Loss: 0.6379 Acc: 0.7219
Epoch 2/9
----------
synthetic_train Loss: 0.5008 Acc: 0.7873
test Loss: 0.4105 Acc: 0.8500
Epoch 3/9
----------
synthetic_train Loss: 0.2421 Acc: 0.9117
test Loss: 0.3958 Acc: 0.8773
Epoch 4/9
----------
synthetic_train Loss: 0.0969 Acc: 0.9678
test Loss: 0.3473 Acc: 0.9094
Epoch 5/9
----------
synthetic_train Loss: 0.0553 Acc: 0.9830
test Loss: 0.4070 Acc: 0.9086
Epoch 6/9
----------
synthetic_train Loss: 0.0426 Acc: 0.9865
test Loss: 0.5260 Acc: 0.8797
Epoch 7/9
----------
synthetic_train Loss: 0.0429 Acc: 0.9869
test Loss: 0.4446 Acc: 0.9023
Epoch 8/9
----------
synthetic_train Loss: 0.0275 Acc: 0.9898
test Loss: 0.3731 Acc: 0.9125
Epoch 9/9
----------
synthetic_train Loss: 0.0253 Acc: 0.9924
test Loss: 0.5404 Acc: 0.8984

Training complete in 3m 29s
Best test Loss: 0.347333


/home/danielkwon/MAS-thesis/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


...

Epoch 0/9
----------
synthetic_train Loss: 1.0117 Acc: 0.5016
test Loss: 0.9289 Acc: 0.5508
Epoch 1/9
----------
synthetic_train Loss: 0.9342 Acc: 0.5504
test Loss: 0.8857 Acc: 0.5664
Epoch 2/9
----------
synthetic_train Loss: 0.8642 Acc: 0.5750
test Loss: 0.8501 Acc: 0.5695
Epoch 3/9
----------
synthetic_train Loss: 0.7604 Acc: 0.6488
test Loss: 0.7214 Acc: 0.6813
Epoch 4/9
----------
synthetic_train Loss: 0.5940 Acc: 0.7449
test Loss: 0.6221 Acc: 0.7438
Epoch 5/9
----------
synthetic_train Loss: 0.4415 Acc: 0.8250
test Loss: 0.7201 Acc: 0.7562
Epoch 6/9
----------
synthetic_train Loss: 0.3140 Acc: 0.8805
test Loss: 0.7229 Acc: 0.7703
Epoch 7/9
----------
synthetic_train Loss: 0.2159 Acc: 0.9252
test Loss: 0.6962 Acc: 0.8070
Epoch 8/9
----------
synthetic_train Loss: 0.1469 Acc: 0.9506
test Loss: 0.6690 Acc: 0.8109
Epoch 9/9
----------
synthetic_train Loss: 0.1189 Acc: 0.9607
test Loss: 0.8337 Acc: 0.7945

Training complete in 3m 26s
Best test Loss: 0.622146


/home/danielkwon/MAS-thesis/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 0/9
----------
synthetic_train Loss: 1.0194 Acc: 0.5082
test Loss: 0.9235 Acc: 0.5477
Epoch 1/9
----------
synthetic_train Loss: 0.9404 Acc: 0.5449
test Loss: 0.8937 Acc: 0.5648
Epoch 2/9
----------
synthetic_train Loss: 0.8845 Acc: 0.5785
test Loss: 0.8298 Acc: 0.5852
Epoch 3/9
----------
synthetic_train Loss: 0.8035 Acc: 0.6305
test Loss: 0.7755 Acc: 0.6250
Epoch 4/9
----------
synthetic_train Loss: 0.6807 Acc: 0.7006
test Loss: 0.6923 Acc: 0.6992
Epoch 5/9
----------
synthetic_train Loss: 0.5226 Acc: 0.7869
test Loss: 0.6954 Acc: 0.7453
Epoch 6/9
----------
synthetic_train Loss: 0.4048 Acc: 0.8418
test Loss: 0.6590 Acc: 0.7680
Epoch 7/9
----------
synthetic_train Loss: 0.2897 Acc: 0.8930
test Loss: 0.6633 Acc: 0.8000
Epoch 8/9
----------
synthetic_train Loss: 0.2166 Acc: 0.9234
test Loss: 0.6422 Acc: 0.7930
Epoch 9/9
----------
synthetic_train Loss: 0.1543 Acc: 0.9479
test Loss: 0.7761 Acc: 0.8078

Training complete in 3m 25s
Best test Loss: 0.642152


/home/danielkwon/MAS-thesis/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 0/9
----------
synthetic_train Loss: 1.0113 Acc: 0.5143
test Loss: 0.9271 Acc: 0.5383
Epoch 1/9
----------
synthetic_train Loss: 0.9376 Acc: 0.5434
test Loss: 0.8699 Acc: 0.5602
Epoch 2/9
----------
synthetic_train Loss: 0.8762 Acc: 0.5818
test Loss: 0.8190 Acc: 0.5883
Epoch 3/9
----------
